In [1]:
import os
import cv2
import numpy as np
import face_recognition
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle

In [2]:
# Step 1: Load and prepare the data
def load_images_from_folder(folder):
    images = []
    labels = []
    for label in os.listdir(folder):
        label_folder = os.path.join(folder, label)
        if os.path.isdir(label_folder):
            for filename in os.listdir(label_folder):
                if not filename.lower().endswith('.avif'):  # Skip .avif files
                    img_path = os.path.join(label_folder, filename)
                    try:
                        img = face_recognition.load_image_file(img_path)
                        images.append(img)
                        labels.append(label)
                    except Exception as e:
                        print(f"Skipping file {img_path}: {e}")
    return images, labels

In [3]:
# Step 2: Face detection and feature extraction
def extract_features(images):
    face_encodings = []
    for img in images:
        # Assuming that each image contains only 1 face.
        bbox = face_recognition.face_locations(img, model='hog')
        if bbox:
            face_encoding = face_recognition.face_encodings(img, known_face_locations=bbox)[0]
            face_encodings.append(face_encoding)
        else:
            # Handle images with no detectable faces
            face_encodings.append(None)
    return face_encodings

In [4]:
# Step 3: Prepare training data
def prepare_training_data(face_encodings, labels):
    # Remove items with None (where no face was detected)
    encodings = [f for f in face_encodings if f is not None]
    labels = [labels[i] for i in range(len(face_encodings)) if face_encodings[i] is not None]
    return encodings, labels

In [5]:
# Step 4: Train a classifier
def train_classifier(encodings, labels):
    clf = svm.SVC(gamma='scale')
    clf.fit(encodings, labels)
    return clf

In [6]:
# Step 5: Evaluate the model
def evaluate_model(clf, X_test, y_test):
    y_pred = clf.predict(X_test)
    return accuracy_score(y_test, y_pred)

In [7]:
# Step 6: Model inference
def recognize_face(clf, face_encoding):
    return clf.predict([face_encoding])[0]

In [34]:
# Function to save the model to a file
def save_model(model, model_path):
    with open(model_path, 'wb') as file:
        pickle.dump(model, file)

In [8]:
# Main workflow
folder = 'FaceEngineDataset/image_database'
images, labels = load_images_from_folder(folder)

In [9]:
face_encodings = extract_features(images)

In [13]:
encodings, labels = prepare_training_data(face_encodings, labels)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(encodings, labels, test_size=0.25, random_state=42)

In [15]:
clf = train_classifier(X_train, y_train)

In [16]:
accuracy = evaluate_model(clf, X_test, y_test)

In [17]:
print(f'Model accuracy: {accuracy}')

Model accuracy: 0.6996197718631179


In [31]:
# Use the trained classifier for new predictions
new_image = face_recognition.load_image_file('FaceEngineDataset/image_database/Elon Musk/download.jpg')
new_face_encoding = face_recognition.face_encodings(new_image)[0]
result = recognize_face(clf, new_face_encoding)
print(result)

Elon Musk


In [35]:
# After training the classifier, save it to a file
model_path = 'face_recognition_model.pkl'  # Define your model save path
save_model(clf, model_path)